In [1]:
import numpy as np
from ipycanvas import Canvas, hold_canvas
import time
from threading import Thread

In [2]:
canvas = Canvas(width=900, height=600)

In [3]:
class FourierDrawer:
    def __init__(self, wave, scale, center=True):
        self.wave = wave
        self.F = np.fft.fft(wave)/scale
        self.F[0] = 0 if center else self.F[0]
        self.dt = 2.0*np.pi/len(wave)
        self.epics = self.calculate_epics()
        self.t = 0
        
    def calculate_epics(self):
        epics = []
        for freq, e in enumerate(self.F):
            amp = np.absolute(e)
            phase = np.angle(e)
            epics.append((freq, amp, phase))
        return sorted(epics, key=lambda x: np.absolute(x[1]), reverse=True)
    
    def tick(self):
        circles = []
        x = 0
        y = 0
        for freq, amp, phase in self.epics:
            phi = freq*self.t + phase
            next_x = amp * np.cos(phi)
            next_y = amp * np.sin(phi)
            circles.append((x, y, amp, next_x, next_y))
            x += next_x
            y += next_y
        self.t += self.dt
        return circles

In [4]:
cwave = []
for i in np.linspace(0, 2*np.pi, 100):
    value = np.cos(i) + 1j*np.sin(i) + np.sin(i*4)
    cwave.append(value)
cwave = np.array(cwave)


In [5]:
class FourierCanvas:
    def __init__(self, canvas, drawer, cx, cy):
        self.canvas = canvas
        self.path = []
        self.drawer = drawer
        self.cx = cx
        self.cy = cy
        
    def draw(self):
        self.canvas.fill_style = "#000000"
        circles = self.drawer.tick()
        with hold_canvas(self.canvas):
            self.canvas.fill_rect(0, 0, self.canvas.size[0], self.canvas.size[1])
            x, y = self.draw_circles(circles)
            self.path.append((x, y))
            if self.drawer.t > 2*np.pi + self.drawer.dt*2:
                self.path = self.path[1:]
            self.draw_path()
    
    def draw_circles(self, circles):
        self.canvas.stroke_style = "#AAAAAA"
        x0, y0, _, _, _ = circles[0]
        for x, y, radius, next_x, next_y in circles:
            self.canvas.begin_path()
            self.canvas.move_to(self.cx+x0, self.cy+y0)
            self.canvas.line_to(self.cx+x0+next_x, self.cy+y0+next_y)
            self.canvas.stroke()
            self.canvas.close_path()
            self.canvas.stroke_arc(self.cx+x0, self.cy+y0, radius, 0, 2*np.pi)
            x0 += next_x
            y0 += next_y
        return x0, y0
    
    def draw_path(self):
        self.canvas.stroke_style = "#FFFFFF"
        self.canvas.line_width = 1.5
        x0, y0 = self.path[0]
        self.canvas.begin_path()
        self.canvas.move_to(self.cx+x0, self.cy+y0)
        for x, y in self.path[1:]:
            self.canvas.line_to(self.cx+x, self.cy+y)
        self.canvas.stroke()
        self.canvas.close_path()
        self.canvas.line_width = 1.0

In [6]:
scale = 400
pi = [10.503452032977105+177.03867896547317j,15.47823282911084+177.40718133578503j,20.473943005378604+177.77723403092622j,25.456893550792444+178.14634156799778j,30.43908808411038+178.5153931041598j,33.84696441056896+175.23279380640582j,36.65246304008075+171.10857975709771j,39.26142598439309+166.83434265487662j,41.75687840647757+162.50880685730363j,44.21843307877759+158.166271425047j,46.708744337328895+153.82460497785118j,49.26042441302248+149.52650758380912j,51.919693809914044+145.27406140538386j,54.69309458439073+141.12055428863744j,57.612715238677+137.0684114034842j,60.70082048697596+133.13025371816033j,63.95350206441642+129.34470176689754j,67.3880684742744+125.71694927817123j,71.007687819234+122.26399188490366j,74.8128682867229+118.9997881126251j,78.76602993210892+115.96131949608724j,82.89347916898683+113.12877884203164j,87.15811538090583+110.52535775624081j,91.55638943719562+108.14583277860633j,96.06253374969941+105.99427869981166j,100.67081441057775+104.06164299070514j,105.36393892516664+102.34286225224767j,110.1369181639514+100.82724930631329j,114.95813922297367+99.51142917042384j,119.8369513287599+98.37969486738655j,124.74092129679613+97.42679313551503j,129.68281399983812+96.63797077075034j,134.6466422379517+96.0048628984915j,139.61606825342707+95.51847020189203j,144.5925259588279+95.16822503212104j,149.59841460221276+94.94420438126866j,154.6031175425881+94.8400531428917j,159.5932796159573+94.84763554291919j,164.59641566509987+94.9598781473347j,169.58154484241263+95.16959542338775j,174.58021939830098+95.47223610717032j,179.56190903314186+95.86066378535476j,184.5393400339068+96.3306000899042j,187.35382404543458+98.76857984476943j,187.25680313139952+103.77344205020225j,187.1356284508736+108.7704456073631j,186.9899971184385+113.75866299616129j,186.81862526087474+118.76409056799969j,186.62196347483194+123.75833900679609j,186.39911596077795+128.75367712138103j,186.1496294909995+133.74878048684403j,185.8730628580956+138.7423294350093j,185.5689874078248+143.73300964997114j,185.23698757754676+148.71951276988148j,184.87572246525727+153.71301263805663j,184.48561835079724+158.69945416300814j,184.06630322599304+163.67754945878116j,183.61742049740255+168.6460191899185j,183.13625495923148+173.62739879032594j,182.62461239110272+178.59604579822923j,182.07956900777006+183.57393494491964j,181.50321606154844+188.53601979063876j,180.89242726750905+193.50371392765632j,180.24808946199653+198.46374944876155j,179.5682523961841+203.42551040147154j,178.8555827421955+208.36516691578544j,178.10657124166812+213.3030096130953j,177.31899381053614+218.24757720485087j,176.4939944173881+223.1859602158811j,175.63482194690562+228.09557897103562j,174.73382581058453+233.01587077757094j,173.7942097212508+237.92399431599023j,172.81162950322664+242.8376582956284j,171.78959543825718+247.7346641377609j,170.72786355306772+252.6131055832541j,169.62189257798715+257.4898542177734j,168.47121187943208+262.36214518411896j,167.27995345592944+267.2091046222612j,166.0387701316282+272.06460948105575j,164.75655820506142+276.8900231853569j,163.42853689991188+281.7008519037824j,162.04960280142055+286.5114103161613j,160.62441254433267+291.30162931020766j,159.14783307133564+296.0852950819494j,157.62487240023756+300.8430573794679j,156.05039353433912+305.58818962546087j,154.42439074461373+310.3174312672741j,152.74696573460312+315.02758758812433j,151.0183329723868+319.71553566635964j,149.23322871093808+324.39264568287916j,147.3975701385185+329.0409190564661j,145.5062266098589+333.6712795810036j,143.55975245041103+338.28002528602246j,141.55887036130298+342.8635642860489j,139.50447992811377+347.4184242904112j,137.39172578762276+351.95381268010874j,135.221765542142+356.46560183273107j,133.00200998157155+360.9378884186266j,130.72209924866908+365.3910496282275j,128.38991987222636+369.8092329490734j,126.0015673375995+374.20005091978896j,123.55338850859944+378.5701322519117j,121.0603570433637+382.8938922512072j,118.50736207711424+387.19903407406605j,115.90410253961838+391.47068444282723j,113.26069114448869+395.6954811034673j,110.5575060945554+399.90787035670934j,107.81190953911184+404.08428906117985j,105.01788289204667+408.2387027198235j,102.18227696174603+412.3665325161616j,99.32429438078935+416.4474756318396j,96.41786231134331+420.52718118792507j,93.48478062215263+424.5846536857627j,90.53630822776451+428.6166224015479j,87.5743453175215+432.6350878875653j,84.59313613618109+436.6651176200357j,81.62115664526813+440.68924660327616j,78.66853119899234+444.71934432646344j,75.74937623770215+448.7674002635171j,72.87400809347733+452.8582389299735j,70.06844319181792+457.0045583940049j,67.37268513469131+461.20939809458537j,64.81221752662239+465.5137402982683j,62.450323918504154+469.9156505848547j,60.339816251320244+474.45263350171683j,58.563825705487346+479.1237070965377j,57.20560171394956+483.93541500954086j,56.352114804919495+488.8566326604421j,56.06585367844414+493.83642862604995j,56.36879172198364+498.8196874019764j,56.95499891439568+503.7967730537742j,57.86551356879985+508.7119917175817j,59.136668494704494+513.5416460927128j,60.77632003464176+518.2648277202243j,62.780156827739376+522.8384802901803j,65.14753219738968+527.2429723311426j,67.86283882074065+531.4392165539532j,70.91761097674167+535.409613784461j,74.2705012180925+539.1040999750594j,77.91972771610678+542.5265184397077j,81.83144748941555+545.6486466016851j,85.96686909354+548.4476385245479j,90.30283965648553+550.9165483348048j,94.83653678519414+553.0571040152179j,99.50477769207963+554.8434745365897j,104.28251637830223+556.2733263550307j,109.16723944978222+557.3484725126408j,114.11386367160492+558.059697416953j,119.10663606699599+558.3580948976314j,124.09207034823156+558.0352697218452j,129.05862699480844+557.4503794578023j,133.9760979510685+556.5783277750779j,138.83230160792348+555.3921244769425j,143.60112292995365+553.8687443372552j,148.2320446307256+552.0002628862662j,152.7089536354747+549.775556224186j,156.993604325632+547.2035397450521j,161.06718506252582+544.2960182450571j,164.89274049277185+541.0936486658399j,168.48512787273128+537.6084439625555j,171.8274067873832+533.8894379465734j,174.92736194979955+529.9710949524086j,177.80282779418346+525.8773262840552j,180.461307422736+521.6469457735577j,182.92797883806483+517.28983916846j,185.20865250346958+512.8458425877782j,187.32539158884234+508.32205932444157j,189.29790162305892+503.7216026789998j,191.13638917712802+499.063376446587j,192.8502915826834+494.36591516245545j,194.44832006691558+489.64736143786047j,195.95522818804386+484.8705750096534j,197.36709352071648+480.08129574273016j,198.69963672707962+475.25919891491105j,199.9578398625098+470.4155584804254j,201.14634045194933+465.561463848372j,202.2752018798514+460.6822499562489j,203.34236701675002+455.812820012793j,204.36249047574762+450.90989685001006j,205.33290792018738+446.00745192588386j,206.26169721369402+441.0855550763608j,207.14610164280202+436.1792917856961j,207.99609374381419+431.25250501917463j,208.81354462844047+426.310338717436j,209.59782540401895+421.37322894143983j,210.35306849785192+416.431470154048j,211.0831217559741+411.47409945486146j,211.78478325869753+406.53761151092203j,212.4662135044153+401.5781999156351j,213.12628028770533+396.6157605264942j,213.76391625033432+391.6712890994059j,214.38666074695917+386.6974504939898j,214.98897432689938+381.7493057449247j,215.57811469284056+376.7778035956993j,216.15272664499258+371.8032632222681j,216.71149695982274+366.847052269967j,217.2592418374204+361.875512791701j,217.79656036452099+356.8907590870297j,218.3200071917646+351.9332859444696j,218.83617511738308+346.9483610765616j,219.3415623158357+341.97669726947004j,219.8386451597625+337.00115474511216j,220.32786585404705+332.0237761494419j,220.80963986589387+327.0465909858968j,221.28435669866118+322.07161520880834j,221.75238065633258+317.1008508217305j,222.21600725492806+312.115130150948j,222.67359683849705+307.13711122160305j,223.1264246902221+302.1579464632718j,223.57574840869114+297.1683096677855j,224.01888910248186+292.2025799755229j,224.46099486730006+287.2072394066013j,224.89836858007752+282.22792424706466j,225.33220118092163+277.25515750666136j,225.7636798277225+272.2790004226461j,226.19399351790645+267.28906083561355j,226.62132855811393+262.30964968967123j,227.04684471888305+257.3304745247578j,227.47171046348242+252.3407979682658j,227.89404523976486+247.36557235489576j,228.31601624069896+242.3820072466276j,228.73776478423594+237.39096195365443j,229.15733909284103+232.4180181447128j,229.5779671756216+227.42725963411434j,229.99767114622247+222.444365144554j,230.4165325771675+217.47049426791182j,230.83677648399618+212.48124129465077j,231.2563496365678+207.50290058391525j,231.67640288250388+202.52363144853737j,232.09701245261908+197.5442662170004j,232.5182476349221+192.56563406792077j,232.941298539137+187.57527371773193j,233.36397696062124+182.6004824957538j,233.78859930353397+177.61540633171452j,234.21408184336556+172.63415005724374j,234.64046222337305+167.65752775487982j,235.07014533065956+162.6587866229316j,235.49963475506527+157.67978691083266j,235.93012530060457+152.70764970031436j,236.36285738746412+147.72910401801047j,236.79788075754283+142.74465999777843j,237.23524126215113+137.75482679644676j,237.67371386806863+132.77447157913895j,238.11329386814012+127.80438536175996j,238.55656311191638+122.81626058756406j,239.000974026815+117.8396010885574j,239.44783377813602+112.86045503266762j,239.8971647162063+107.87932277604313j,240.3489857685654+102.89670345200975j,240.8033124983845+97.91309494037586j,243.9222430139877+95.86892099999999j,248.92858421515544+95.86892099999999j,253.93628532681848+95.86892100000001j,258.9234491496946+95.868921j,263.92011632056193+95.868921j,268.92408802304107+95.868921j,273.92748301952884+95.868921j,278.9268192054692+95.868921j,283.9235497975714+95.868921j,288.93560397017274+95.868921j,293.9241183008599+95.868921j,298.9228847768565+95.868921j,303.92464293343136+95.86892099999999j,308.9233956920527+95.868921j,313.9356543406838+95.868921j,318.93073398317057+95.868921j,323.91976659215834+95.86892100000001j,328.92813682284657+95.868921j,333.927327716041+95.868921j,338.93534382966067+95.868921j,343.92326894801903+95.868921j,348.92552707565426+95.868921j,353.9243694801314+95.868921j,358.9302643119417+95.868921j,363.92171156857705+95.868921j,365.6262146122886+98.98809718704257j,365.32501877053835+103.97451050540016j,365.0104379791646+108.9608577101494j,364.6829603105234+113.94709812854674j,364.3416016942179+118.95440813155824j,363.9897448044599+123.94031207155402j,363.6264573314069+128.9259870338917j,363.252227347415+133.91139234582766j,362.86754292484045+138.8964873346184j,362.47289213603943+143.8812313275203j,362.06876305336823+148.86558365178988j,361.65564374918296+153.8495036346835j,361.23402229583996+158.83295060345765j,360.80530904627193+163.80528247372908j,360.36910074970206+168.7770624270313j,359.9249301976372+173.7588489352076j,359.47420770893575+178.7399999113656j,359.0174213559539+183.72047468276185j,358.5550592110479+188.70023257665267j,358.08760934657397+193.67923292029454j,357.6155598348882+198.6574350409439j,357.13939874834693+203.63479826585723j,356.6596141593064+208.61128192229089j,356.17669414012266+213.58684533750133j,355.6911267631521+218.561447838745j,355.2034001007508+223.53504875327835j,354.71295951878665+228.5181861803061j,354.2213317989825+233.50023597510545j,353.7300537995594+238.47058322484284j,353.2385708810342+243.43976602382006j,352.7452822057397+248.42888139676342j,352.25485799076057+253.3956078878232j,351.76361699857415+258.3821744608429j,351.2762120930185+263.34628125997244j,350.7889877328212+268.3301349538047j,350.3055395638365+273.3020131538425j,349.82532396323927+278.27243541237215j,349.3478247755862+283.2519089104332j,348.87455950922566+288.2298379769917j,348.40602249377594+293.2061814178649j,347.94270805885543+298.18089803887017j,347.48318442759+303.17501527822816j,347.03182551314205+308.146347346896j,346.5853051212468+313.13698260417067j,346.14788743308884+318.1047646448243j,345.71636440038856+323.0917517431707j,345.2931058047287+328.0768484135757j,344.8786122862428+333.0600129359799j,344.47338448506446+338.0412035903238j,344.07792304132727+343.0203786565477j,343.6911257624672+348.0184924937853j,343.3167454477429+352.9935022785429j,342.95212799999854+357.98734911317337j,342.60011151739127+362.9685103546699j,342.26042172111084+367.94743757114384j,341.9315398961038+372.95551293721235j,341.6174552883431+377.94030115063117j,341.31727140089674+382.9227190827647j,341.03032560778945+387.9236437264715j,340.7589750652161+392.911638344932j,340.5031143294441+397.8971251607703j,340.2627679183774+402.89050535579736j,340.0381013929906+407.9021581208134j,339.83150218782583+412.8902767482686j,339.64175952901377+417.8965596424376j,339.470969818875+422.87924484469437j,339.3182219445865+427.87998529132034j,339.1847888363139+432.87788383224114j,339.0711903875107+437.8728971429521j,339.04714696512764+442.87653106059656j,339.26062378563097+447.8793711159578j,339.61255488819177+452.85410807907584j,340.111756049253+457.83363651521967j,340.7639078044034+462.7968564426391j,341.5731223511948+467.7237003704534j,342.54966496688075+472.6308855377567j,343.6977441596933+477.49762320759737j,345.0232590759815+482.3154579958575j,346.5319337405183+487.07572708593705j,348.2333494693185+491.780071048186j,350.12951015666397+496.4081619597109j,352.22550835598474+500.9506758209763j,354.5261916620196+505.3980704557461j,357.025154578363+509.72256951661944j,359.7245815772313+513.91655621135j,362.63879727071475+517.9887012263289j,365.74538911594675+521.897440485694j,369.05746125384155+525.6519600714564j,372.5617433955301+529.2301244624108j,376.2421036929978+532.613039151437j,380.0953667434458+535.7973954475257j,384.1019795655294+538.7684464671422j,388.2736305658509+541.5359538925966j,392.57204016147193+544.0771578001381j,397.0083166461251+546.4026744388998j,401.54038861806896+548.495763473051j,406.1790850991446+550.3679156804335j,410.8985393547298+552.0146709268035j,415.6910877572489+553.44040027286j,420.54932382989654+554.6494692184851j,425.44540929353104+555.6425027095503j,430.37130773529054+556.4263526037597j,435.34076945971697+557.0099095701937j,440.32574833746867+557.396222001617j,445.3192377759159+557.5920294544297j,450.32318754941616+557.6830403838447j,455.3241528782715+557.6129568552835j,460.3084187546273+557.3319949691037j,465.2861023713932+556.8311302808977j,470.2331208525687+556.1045327154841j,475.14291534596265+555.1463139358417j,479.9936132966199+553.9553399255891j,484.7949091510037+552.5241466832314j,489.51146542472986+550.8597092802379j,494.1255730070561+548.9687266278686j,498.64842782835956+546.847408513459j,503.06334215070206+544.5047154952754j,507.3681134578762+541.9445618902246j,511.5362375479772+539.1882751891281j,515.5682882528424+536.2438424332121j,519.4761807392892+533.1099297478363j,523.2374974772473+529.8133728844974j,526.8541919908012+526.3640615572799j,530.328217804036+522.7718854802686j,533.6615284410361+519.0467343675482j,536.8560774258867+515.1984979332034j,539.9138182826724+511.2370658913188j,542.8367045354779+507.1723279559792j,545.6186531923158+503.02656747673643j,548.2704199771773+498.79774358555926j,550.8012188606766+494.4828316883885j,553.2049657113189+490.1041144077058j,555.4901526037356+485.6583358554301j,557.663823806726+481.1417774019835j,559.7141725366503+476.5905946199388j,561.6541326738877+471.9881319119512j,563.4887351484649+467.330949701339j,565.2121523803053+462.6428411526802j,566.8376983623589+457.8944393413099j,568.3501834877634+453.13751463608276j,569.7610443491019+448.34425218870734j,571.0709494032197+443.5150315807551j,572.2762865347615+438.6648718492637j,573.3790468470609+433.7846902369431j,574.3763051505055+428.88124162187296j,575.2639771296876+423.9644172573303j,576.0385513956021+419.0360485771687j,576.696772445898+414.0812593123734j,577.2301304418802+409.0964059212687j,577.6234427761725+404.1128267849844j,577.8607419600264+399.1168446494384j,576.767539253769+395.2770905376434j,571.761812336988+395.2770928722963j,566.7602067842729+395.277095205027j,561.765573774788+395.2770975345058j,556.7661931298612+395.2770998661988j,556.1616221296551+399.9802589136523j,555.6367978032895+404.944492131226j,554.9004252012146+409.89790978896536j,553.9378505137282+414.80821782451756j,552.7325507182045+419.6564380663589j,551.2631635217252+424.4346072871754j,549.5099176177272+429.1192931875593j,547.4577715604446+433.6741050326877j,545.0874182748815+438.0739796905932j,542.3857869810677+442.282293634595j,539.3574007693849+446.2451596439571j,535.9899003508648+449.94159844197367j,532.3074052461604+453.31907949212416j,528.3261619655333+456.3510223790448j,524.0900057386376+459.00539881407485j,519.6321063213135+461.27448265338614j,515.0039291066269+463.1537822272883j,510.23986071958694+464.6569755489238j,505.3769300827154+465.8020245002616j,500.44026768930115+466.6128761857397j,495.46952313851955+467.11301011623686j,490.4638567763696+467.25880515979134j,485.4794649313556+467.05380246623173j,480.4943946708882+466.5391388178408j,475.57645496134177+465.68309931351183j,470.73599618117197+464.44850320116564j,466.00256097937984+462.7995326076744j,461.4624687298565+460.73008716994275j,457.1314745237987+458.21988463524394j,453.0857162360505+455.2985092061177j,449.3465075168153+451.9873542541663j,445.937270146119+448.32969981353756j,442.87024156047835+444.3823127615846j,440.13917438349955+440.1998582172382j,437.72475337157977+435.8284739572644j,435.59928310141174+431.3001148590028j,433.73691307264414+426.6447862986994j,432.124832020882+421.9264082287767j,430.72183287484853+417.12002777754907j,429.513490160477+412.2672553193754j,428.47665440502925+407.37095419604515j,427.5950116269781+402.4505993922566j,426.85191348940106+397.51377063150846j,426.23018385444357+392.5437749748099j,425.7222914809902+387.58709599028526j,425.31228773545956+382.6005676131002j,424.9926746242219+377.6057050987116j,424.7552307912992+372.6124955513605j,424.5916041312664+367.6059217349363j,424.495795532982+362.61853086306786j,424.46091234371437+357.62086095159856j,424.48177284505135+352.6191848897648j,424.5534626899274+347.61949356539714j,424.6713262783451+342.6275044062718j,424.8319243984217+337.6218737172429j,425.0305504228982+332.63393921138385j,425.2652975783965+327.62753930452277j,425.53169075006304+322.63382208109067j,425.82727156327707+317.6429353541637j,426.149130835004+312.6584445486352j,426.4955183129534+307.66949562567146j,426.8639987958111+302.67900349840613j,427.2510755360833+297.7043045230465j,427.65675668069974+292.7191908187376j,428.07906193904876+287.7259504525312j,428.51343667811204+282.75649240659465j,428.961836904271+277.7687529248249j,429.4198040066395+272.79459351162734j,429.8883647908017+267.8060826281702j,430.3630172661826+262.8353327452381j,430.8450329872017+257.85395555399316j,431.33140686866886+252.8789350571664j,431.8221409110479+247.8966768315988j,432.3142678580168+242.92422886850557j,432.8063579871481+237.9633148485147j,433.300120762912+232.98443523799483j,433.7926779442629+228.00454779831503j,434.28271001669054+223.02511177572742j,434.768922234481+218.0475614080447j,435.2500444263768+213.07330611711106j,435.72632905183946+208.08793802818167j,436.19353675141144+203.1243738931958j,436.6548888465237+198.13649687878444j,437.1061914287797+193.15722857368763j,437.54627106914523+188.18788115432545j,437.97668178436265+183.1979474997438j,438.39342945013004+178.22045123692567j,438.7966537492532+173.2407449978055j,439.18632140392367+168.24415314925108j,439.55869779977934+163.26374172712426j,439.91490434497155+158.2689329139676j,440.251419553224+153.29278138872957j,440.56913568771927+148.30473655374118j,440.86659519369226+143.30609191857417j,441.14148034218556+138.31400122916475j,441.3932753209009+133.31391635788574j,441.61978526376816+128.3229695486269j,441.8203137574172+123.32669072174818j,441.9933037515371+118.32647049227293j,442.1371727163792+113.32372423186537j,442.2500089116094+108.33550509558145j,442.33088950744855+103.332000835562j,442.3777512039375+98.33036278787412j,446.4389547386366+97.42333837647728j,451.44642821093106+97.4516291933171j,456.4404038429989+97.47984375115573j,461.4427437046868+97.50810556453939j,466.438331082255+97.53632922828682j,471.43802364253577+97.56457608516206j,476.4532339772426+97.59291061309864j,481.4493035227775+97.62113700095951j,486.44632409019397+97.6493687618265j,491.44654029180566+97.67761857712796j,496.4375510261842+97.70581638412848j,501.4371158305407+97.73406251921868j,506.45523412925223+97.76241347633407j,511.44189095173897+97.79058668495688j,516.4470550410306+97.81886445443223j,521.4460660294505+97.84710746061813j,526.4377037896384+97.87530881013814j,531.4418369446934+97.90358075512498j,536.4488320585831+97.93186886937437j,541.4426836422201+97.96008272637425j,546.4418850091365+97.98832680814512j,551.4453542018165+98.01659500193128j,556.4439453435677+98.04483563610157j,561.4531256554458+98.07313609610503j,566.4520854115191+98.1013788128426j,571.4514738419325+98.1296239514696j,576.4443414974621+98.1578322495524j,577.8890940997044+94.63990491861955j,577.8447812676538+89.63255582281724j,577.8006515625+84.6459000625j,577.7563798351043+79.64319579182944j,577.7120669240129+74.6358377643957j,577.6678408190837+69.63828883149084j,577.6236931379539+64.6496017862552j,577.5794136876872+59.64602483132602j,577.5351816032074+54.64780020931539j,577.4908369346717+49.636853591337925j,577.446641590069+44.64278057468236j,577.4023738630957+39.64052835165295j,577.3581230536043+34.64018780372066j,577.3139114474852+29.644277236040594j,577.2696047816635+24.637624923960416j,577.2253883285646+19.641166647665525j,577.1811312235924+14.64011471054456j,575.1586435214482+11.626721362585874j,570.1601969045964+11.601980050118309j,565.1572394845737+11.576646036811814j,560.1630238507148+11.550808411563285j,555.1529149661371+11.52436093280555j,550.1651039833672+11.497529536795513j,545.1627027410282+11.470141461682394j,540.1711262823595+11.442360930199431j,535.1664361339016+11.414081361347051j,530.1616213141822+11.385402765809657j,525.1693170646727+11.35642859124267j,520.1542168400645+11.326983990329593j,515.1529250023877+11.297315616383356j,510.16582116068844+11.26746231687198j,505.17002753104384+11.23732328986454j,500.1664591441765+11.206940478552918j,495.15603184948367+11.176358183470796j,490.16237308013723+11.14576246422825j,485.1633231086064+11.115061361219755j,480.15979647021186+11.084305922473321j,475.16378873099825+11.053617758353608j,470.1649486796551+11.022983520886116j,465.15329291025523+10.992395419630995j,460.1624223339881+10.962113695801632j,455.17126984774814+10.932067568837374j,450.15951148929497+10.902197704201265j,445.17070939950315+10.872827902005461j,440.1634933139462+10.843782141621782j,435.1600032251951+10.815262366266644j,430.1611438270052+10.787347779290895j,425.1678175288102+10.760120730015338j,420.1608303349523+10.733561791912713j,415.1615577277027+10.707873734317214j,410.17089527482597+10.683149624292648j,405.17022138981605+10.659395370687744j,400.161008963601+10.636727271907715j,395.16385327450973+10.615346498430332j,390.1607118327328+10.595289008358607j,385.17178215679695+10.576753805331153j,380.16088023857634+10.559738991689661j,375.1667281818323+10.544514692233422j,370.1720620360368+10.531163781643501j,365.1604156294968+10.51980538926803j,360.169302789731+10.510684075950406j,355.16452819925047+10.503904794803075j,350.1654709124933+10.499681729316581j,345.15646733806653+10.498199186760326j,340.1564818505364+10.499669635917112j,335.16695117532345+10.504293604869874j,330.1728167983944+10.512311556101217j,325.15985025527897+10.524019155711946j,320.162862906452+10.539602974298324j,315.1674080785454+10.559369454996759j,310.15987987964934+10.583687778678849j,305.158375121585+10.612801065403499j,300.1649153878039+10.647021590251395j,295.1665194311799+10.686802425194337j,290.1658413501699+10.732530467535325j,285.1655286874932+10.784610314523922j,280.16822033202675+10.843464796590562j,275.16243441671685+10.90973308656756j,270.1653435666427+10.98371724276884j,265.16589047132817+11.066137735634284j,260.1673399152837+11.157561683456269j,255.17294318034774+11.258582724913751j,250.17296044157789+11.370125440401162j,245.1713004273634+11.492911400200848j,240.17185609458448+11.627697609185088j,235.17849986766328+11.775277716813648j,230.17100652371977+11.937297436563206j,225.19001479228982+12.11349254710057j,220.1923638206271+12.306555919191705j,215.18397169801042+12.517728454539206j,210.19287350121138+12.747257390061772j,205.20181235021593+12.997447787257268j,200.21660349847866+13.269746262193074j,195.22236376614873+13.56695858005752j,190.22630796152336+13.890929889440415j,185.23559507184655+14.243611880582884j,180.25731063923004+14.627067264545683j,175.2798525143964+15.04510476340858j,170.29359776269396+15.502047692885384j,165.32600651643943+15.999123170047547j,160.3512621924493+16.54314513886422j,155.38061206913156+17.137904636609374j,150.42513413779244+17.787443523039258j,145.48031313612788+18.498381988755618j,140.54357122836046+19.278169011789846j,135.61424674228587+20.13517755088776j,130.70725839568848+21.07605208802385j,125.81081679912249+22.113772143102175j,120.94226566097467+23.257039041113796j,116.10630918825422+24.51851958065946j,111.3091419736869+25.91240189057835j,106.54743656961708+27.45829371171034j,101.8417935766864+29.170723297715085j,97.23074058760797+31.057004603790308j,92.69173326725127+33.150447897541234j,88.24344594485702+35.4710376261531j,83.95036336596522+38.00996716264717j,79.80137993670222+40.79555269924951j,75.82916561556667+43.82553448920351j,72.0428731337222+47.105490858378865j,68.47056469532109+50.61310142150595j,65.1290118698537+54.31757620846387j,61.99309368066439+58.222446993684706j,59.0799102571812+62.27000232929456j,56.34957023399457+66.46996370050269j,53.80447005648833+70.76740607806391j,51.41737232261386+75.15050385640025j,49.155202066305336+79.6255666420907j,47.01504815854625+84.14411532140383j,44.973716869797116+88.70122517285706j,43.01107115634686+93.29338889609315j,41.10952935870389+97.91852002319078j,39.25362157237951+102.57595022386981j,37.44286244418738+107.23196644263069j,35.661011427760876+111.89730167534981j,33.892877350987725+116.58557254045883j,32.142570579961344+121.26300965512058j,30.397055044714932+125.94463504843789j,28.653298952120082+130.62177854914484j,26.8990443173581+135.3129278388433j,25.142622390644455+139.98364099863028j,23.367321380586375+144.66800289335146j,21.577410685607276+149.34581554226563j,19.778450967255324+153.99559247158726j,17.95339407545719+158.65537010237443j,16.108046119573494+163.3049737839358j,14.243042633297419+167.93880076381254j,12.352788702735303+172.56738508600085j]
pi = pi[0::2]

In [9]:
drawer = FourierDrawer(pi, scale)
fc = FourierCanvas(canvas, drawer, canvas.size[0]/2, canvas.size[1]/2)

for i in range(len(pi)*5):
    fc.draw()

In [8]:
canvas

Canvas(height=600, width=900)